In [1]:
import pandas as pd
import json

In [10]:
import requests
url = "https://ero.betfair.com/www/sports/exchange/readonly/v1/bymarket?_ak=nzIFcwyWhrlwYMrh&currencyCode=GBP&locale=en_GB&marketIds=1.128151441&rollupLimit=10&rollupModel=STAKE&types=MARKET_STATE,RUNNER_STATE,RUNNER_EXCHANGE_PRICES_BEST,RUNNER_DESCRIPTION"
url2 = "https://ero.betfair.com/www/sports/exchange/readonly/v1/bymarket?_ak=nzIFcwyWhrlwYMrh&alt=json&currencyCode=GBP&locale=en_GB&marketIds=1.170917778,1.128988348&rollupLimit=10&rollupModel=STAKE&types=MARKET_STATE,RUNNER_STATE,RUNNER_EXCHANGE_PRICES_BEST"

payload = {}
headers = {
  'Connection': 'keep-alive',
  'Accept': 'application/json, text/plain, */*',
  'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36',
  'DNT': '1',
  'Origin': 'https://www.betfair.com',
  'Sec-Fetch-Site': 'same-site',
  'Sec-Fetch-Mode': 'cors',
  'Sec-Fetch-Dest': 'empty',
  'Referer': 'https://www.betfair.com/',
  'Accept-Language': 'en-GB,en-US;q=0.9,en;q=0.8',
  'Cookie': '__cfduid=d3013254245c8d86c0aae9e7a0fe0cd661604447496; wsid=8246db31-1e2f-11eb-b7f4-fa163ea4ad04; vid=fa6d8bae-3b04-4a6b-b4f4-76df1b3615de; bfsd=ts=1604447498682|st=p; storageSSC=lsSSC%3D1; xsrftoken=a0a7a690-1e2f-11eb-9f1a-fa163e8a430a; PI=3013; StickyTags=rfr=3013; TrackingTags=; pi=partner3013; rfr=3013; exp=ex; language=en_GB; betexPtk=betexCurrency%3DGBP~betexTimeZone%3DEurope%2FLondon~betexRegion%3Dundefined~betexLocale%3Den_GB; betexPtkSess=betexCurrencySessionCookie%3DGBP~betexLocaleSessionCookie%3Den_GB~betexRegionSessionCookie%3DGBR'

}

response = requests.request("GET", url, headers=headers, data = payload)

In [11]:
#unpack the data from the json - odds snapshot
df=pd.json_normalize(json.loads(response.text),record_path=["eventTypes","eventNodes","marketNodes","runners"],meta=["eventTypes"], errors="ignore")


In [12]:
#unpack the timestamp and metadata
df2=df.eventTypes.apply(pd.Series).eventNodes.apply(pd.Series)[0].apply(pd.Series).marketNodes.apply(pd.Series)[0].apply(pd.Series).state.apply(pd.Series)

In [13]:
#combine metadata with odds data
final_df=pd.concat([df,df2.reindex(df.index)],axis=1)

In [14]:
final_df

,selectionId,handicap,description.runnerName,state.sortPriority,state.lastPriceTraded,state.totalMatched,state.status,exchange.availableToBack,exchange.availableToLay,state.removalDate,...,numberOfWinners,numberOfRunners,numberOfActiveRunners,lastMatchTime,totalMatched,totalAvailable,crossMatching,runnersVoidable,version,status
0,6816445,0.0,Joe Biden,1,1.12,0.0,ACTIVE,"[{'price': 1.12, 'size': 61436.45}, {'price': ...","[{'price': 1.13, 'size': 46208.91}, {'price': ...",NaN,...,1,124,123,2020-11-05T00:26:05.657Z,4.722070e+08,3.669343e+07,False,False,3450999831,OPEN
1,10874213,0.0,Donald Trump,2,9.20,0.0,ACTIVE,"[{'price': 9.2, 'size': 124.71}, {'price': 9.0...","[{'price': 9.4, 'size': 156.73}, {'price': 9.6...",NaN,...,1,124,123,2020-11-05T00:26:05.657Z,4.722070e+08,3.669343e+07,False,False,3450999831,OPEN
2,4296631,0.0,Mike Pence,3,1000.00,0.0,ACTIVE,"[{'price': 1000.0, 'size': 9424.71}, {'price':...",NaN,NaN,...,1,124,123,2020-11-05T00:26:05.657Z,4.722070e+08,3.669343e+07,False,False,3450999831,OPEN
3,3110151,0.0,Tim Kaine,4,1000.00,0.0,ACTIVE,"[{'price': 1000.0, 'size': 11746.76}, {'price'...",NaN,NaN,...,1,124,123,2020-11-05T00:26:05.657Z,4.722070e+08,3.669343e+07,False,False,3450999831,OPEN
4,7438258,0.0,Cory Booker,5,1000.00,0.0,ACTIVE,"[{'price': 1000.0, 'size': 9028.14}, {'price':...",NaN,NaN,...,1,124,123,2020-11-05T00:26:05.657Z,4.722070e+08,3.669343e+07,False,False,3450999831,OPEN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,30274172,0.0,Jo Jorgensen,120,1000.00,0.0,ACTIVE,"[{'price': 1000.0, 'size': 9257.64}, {'price':...",NaN,NaN,...,1,124,123,2020-11-05T00:26:05.657Z,4.722070e+08,3.669343e+07,False,False,3450999831,OPEN
120,35709762,0.0,Howie Hawkins,121,1000.00,0.0,ACTIVE,"[{'price': 1000.0, 'size': 9424.36}, {'price':...",NaN,NaN,...,1,124,123,2020-11-05T00:26:05.657Z,4.722070e+08,3.669343e+07,False,False,3450999831,OPEN
121,36024818,0.0,Mark Esper,122,1000.00,0.0,ACTIVE,"[{'price': 1000.0, 'size': 9471.79}, {'price':...",NaN,NaN,...,1,124,123,2020-11-05T00:26:05.657Z,4.722070e+08,3.669343e+07,False,False,3450999831,OPEN
122,36502219,0.0,Patrick Leahy,123,1000.00,0.0,ACTIVE,"[{'price': 1000.0, 'size': 10539.37}, {'price'...",NaN,NaN,...,1,124,123,2020-11-05T00:26:05.657Z,4.722070e+08,3.669343e+07,False,False,3450999831,OPEN


In [15]:
final_df.to_csv("data.csv",mode='a',header=False)